# 政务文本分类
## 零、数据加载

引入第三方工具包

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import openpyxl  # 读取excel的工具
import jieba  # 中文分词工具
import sqlite3  # 数据库引擎
import nltk  # 英文nlp工具
import gensim  # 词向量化工具
import sklearn  # sci-kit learn机器学习工具

import matplotlib.pyplot as plt  # 绘图工具
plt.rcParams['font.sans-serif']=['SimHei'] #用来正常显示中文标签
plt.rcParams['axes.unicode_minus']=False #用来正常显示负号

path_cc = "/kaggle/input/citizen-comment/citizen-comments-v2/"  # 数据集路径

表格读取方法

In [2]:
def read_xl_by_line(path: str, skip_first_line=True):
    """
    读取表（通用）
    :param path: excel文件路径
    :param skip_first_line: 是否跳过首行
    :return: 列表，元素为n元组
    """
    res_rows = []
    wb = openpyxl.load_workbook(path, read_only=True)
    ws = wb[wb.sheetnames[0]]

    if skip_first_line:
        row = ws[2:ws.max_row]  # 跳过第一行（表头）
    else:
        row = ws[1:ws.max_row]

    for r in row:
        # 将openpyxl的内置对象转化为元组
        row_tuple = tuple(map(lambda x: x.value, r))
        if row_tuple[0] is None:  # 到达空行时退出
            break
        res_rows.append(row_tuple)
    return res_rows

读取四个附件表格到列表对象

In [3]:
# 读取四个附件表
labels = read_xl_by_line(path_cc + "xls/e1.xlsx")
comments = read_xl_by_line(path_cc + "xls/e2.xlsx")
comments_with_likes = read_xl_by_line(path_cc + "xls/e3.xlsx")
comments_with_reply = read_xl_by_line(path_cc + "xls/e4.xlsx")

## 一、预处理
### 1.1 中文分词
这里使用jieba（结巴）中文分词工具

+ 附件一（标签体系）不需要分词
+ 附件二、三中的“主题”、“详情”需要处理
+ 附件四中的“主题”、“详情”、“答复意见”需要处理

In [4]:
# 效果试验
words = list(jieba.cut(comments[1][4]))  # 默认模式，即精准模式

# 尝试用nltk中的FreqDist进行词典生成
fd = nltk.FreqDist(words)
comment_text = nltk.Text(fd)
comment_text

Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
Loading model cost 1.559 seconds.
Prefix dict has been built successfully.


<Text: 
 	 位于 书院 路 主干道 的 在水一方...>

为了方便留言的处理，采用面向对象的思想，将附件中的每一行抽象成为一个“留言”对象

后面用到的“分词”、“去停用词”等方法也写进了类中

In [5]:
# 建立“留言”对象，映射表中的每一行（留言）
# 同时储存分词、标签等处理结果
class Comm(object):
    """留言"""
    def __init__(self):
        """构造方法"""
        self.comm_id = None  # 留言编号
        self.user_id = None  # 用户编号
        self.topic = None  # 留言主题
        self.date = None  # 留言时间
        self.detail = None  # 留言详情
        self.fir_lev_label = None  # 一级标签
        self.likes = None  # 点赞数
        self.treads = None  # 反对数
        self.reply = None  # 答复
        self.reply_date = None  # 答复时间
        
        self.seg_topic = None  # 分词后的留言主题
        self.seg_detail = None  # 分词后的留言详情
        self.seg_reply = None  # 分词后的留言回复classmethod
    
    def load_from_row(self, row):
        """从行元组加载实例"""
        if len(row) == 6:
            # 六元组是附件二的格式
            self.comm_id, self.user_id, self.topic, \
                self.date, self.detail, self.fir_lev_label = row
        if len(row) == 7:
            # 附件三和附件四都是七元组
            if isinstance(row[5], int):
                # 附件三的第六列是点赞数，是整型数据
                self.comm_id, self.user_id, self.topic, \
                self.date, self.detail, self.likes, self.treads = row
            else:
                # 附件四
                self.comm_id, self.user_id, self.topic, \
                self.date, self.detail, self.reply, self.reply_date = row
                
    def cut(self, cut_all=False, stop_words_lt=None):
        """分词并去停用词"""
        # 跳过无意义的行
        if self.comm_id is None:
            return
        
        # 分别对主题、详情、回复字段分词
        self.seg_topic = jieba.lcut(self.topic, cut_all=cut_all)
        self.seg_detail = jieba.lcut(self.detail, cut_all=cut_all)
        if self.reply is not None:
            self.seg_reply = jieba.lcut(self.reply, cut_all=cut_all)
        
        # 去停用词
        if stop_words_lt is not None:
            # 扫描主题
            self.seg_topic = [word for word in self.seg_topic if word not in stop_words_lt]
            # 扫描详情
            self.seg_detail = [word for word in self.seg_detail if word not in stop_words_lt]
            # 扫描回复
            if self.reply is not None:
                self.seg_reply = [word for word in self.seg_reply if word not in stop_words_lt]
                
    def get_vec(self, model):
        """计算词向量"""
        vec = []
        topic_vec = []
        detail_vec = []
        
        for word in self.seg_topic:
            try:
                word_vec = model[word]
            except KeyError:  # 跳过未登录词
                continue
            topic_vec.append(word_vec)
        for word in self.seg_detail:
            try:
                word_vec = model[word]
            except KeyError:  # 跳过未登录词
                continue
            detail_vec.append(word_vec)
        vec.append(topic_vec)
        vec.append(detail_vec)      

        if self.seg_reply is not None:
            reply_vec = []
            for word in self.seg_reply:
                try:
                    word_vec = model[word]
                except KeyError:  # 跳过未登录词
                    continue
                reply_vec.append(word_vec)
            vec.append(reply_vec)
        return np.array(vec, dtype=np.float64)

### 1.2 去停用词
这里使用的停用词表来自哈工大停用词表, 百度停用词表, 四川大学机器智能实验室停用词库的整合，共有2312个词

In [6]:
# 定义加载停用词表的方法
def load_word_list(filepath):  
    stopwords = [line.strip() for line in open(filepath, 'r', encoding='utf-8').readlines()]
    return stopwords 

# 加载停用词到列表
stop_words_lt = load_word_list(path_cc + "special-words/stop_words.txt")

stop_words_lt[:10]  # 展示前十个

['"', '#', '$', '&', "'", '(', ')', '*', '+', ',']

~~考虑到本题中的地名（大写字母加数字的形式）没有意义，把他们也放到停用词~~

地名（城市名、市区名等）可能与题目2的热度问题有关，暂且不去除

In [7]:
# 大写字母表
uppercase = [chr(ch) for ch in range(ord("A"), ord("Z")+1)]
# 大写字母加一位数字的形式
city_names = [letter+num 
              for letter in uppercase
              for num in [chr(n) for n in range(ord("0"), ord("9")+1)]]

# 将它们放入停用词表（4.11：暂时不添加）
# stop_words_lt.extend(uppercase + city_names)

In [8]:
# 从行生成Comm字典的方法
def generate_comm_dict(row_lt, cut_all=False, stop_words_lt=None):
    """从数据行生成key为留言编号，value为Comm对象的字典
    并对字典中的Comm对象执行分词、去停用词等预处理"""
    comm_dict = {}
    for row in row_lt:
        c = Comm()
        c.load_from_row(row)
        c.cut(cut_all=cut_all, stop_words_lt=stop_words_lt)
        comm_dict[row[0]] = c
    return comm_dict

考虑到切分歧义，这里尝试使用jieba的全模式

In [9]:
# 为附件三、四执行同样的映射和预处理
comm_dict_2 = generate_comm_dict(comments, cut_all=True, stop_words_lt=stop_words_lt)
comm_dict_3 = generate_comm_dict(comments_with_likes, cut_all=True, stop_words_lt=stop_words_lt)
comm_dict_4 = generate_comm_dict(comments_with_reply, cut_all=True, stop_words_lt=stop_words_lt) 

查看一个位于附件三的评论的主题，可以看到已经完成分词和去停用词

In [10]:
comm_dict_3[289408].seg_topic

['A', '市', '人才', 'app', '申请', '购房', '补贴', '通不过']

### 1.3 向量化（特征工程）
使用word2vec训练词嵌入（word embedding）模型

In [11]:
# 获取表2中标注的七个一级标签
labels_in_e2 = list(set([c.fir_lev_label for c in comm_dict_2.values()]))
labels_in_e2

['教育文体', '城乡建设', '商贸旅游', '劳动和社会保障', '交通运输', '环境保护', '卫生计生']

In [12]:
# 把表2中的“主题”和“详情”文本“处理成gensim的lineSentence形式，
# 并输出到文本文件
text_file = open("./e2_line_sents.txt", "w", encoding="utf8")
for c in comm_dict_2.values():
    text_file.write(" ".join(c.seg_topic) + "\n")
    text_file.write(" ".join(c.seg_detail) + "\n")
text_file.close()

模型参数表

+ LineSentence(inp)：格式简单：一句话=一行; 单词已经过预处理并被空格分隔。
+ size：是每个词的向量维度； 
+ window：是词向量训练时的上下文扫描窗口大小，窗口为5就是考虑前5个词和后5个词； 
+ min-count：设置最低频率，默认是5，如果一个词语在文档中出现的次数小于5，那么就会丢弃； 
+ workers：是训练的进程数（需要更精准的解释，请指正），默认是当前运行机器的处理器核数。这些参数先记住就可以了。
+ sg ({0, 1}, optional) – 模型的训练算法: 1: skip-gram; 0: CBOW
+ alpha (float, optional) – 初始学习率
+ iter (int, optional) – 迭代次数，默认为5

In [13]:
from gensim.models import Word2Vec
from gensim.models.word2vec import LineSentence

# 使用表2中的语料建立模型，使用CBOW
word2vec_model = Word2Vec(LineSentence("./e2_line_sents.txt"), size=400, window=5, sg=0, min_count=5)  # 训练模型
# word2vec_model.save("./word2vec_model")
word2vec_model.wv.save_word2vec_format("./word2vec_e2_model", binary=False)  # 保存模型

In [14]:
# 测试模型
# 输出与某个词相近的10个词
test_words = "机场"
word2vec_model.wv.most_similar(test_words)

[('K', 0.9999104142189026),
 ('市', 0.9999015927314758),
 ('换乘', 0.9998969435691833),
 ('车站', 0.9998964667320251),
 ('城市', 0.9998923540115356),
 ('区', 0.9998903870582581),
 ('线', 0.9998894333839417),
 ('A', 0.9998855590820312),
 ('火车', 0.9998847842216492),
 ('建议', 0.9998830556869507)]

考虑到表2、3、4中都包含留言文本，尝试使用三个表的文本（而不是只用表2）来训练词嵌入模型

In [15]:
# 将三个表的语料输出到文本文件
text_file = open("./line_sents.txt", "w", encoding="utf8")
for d in (comm_dict_2, comm_dict_3, comm_dict_4):
    for c in d.values():
        text_file.write(" ".join(c.seg_topic) + "\n")
        text_file.write(" ".join(c.seg_detail) + "\n")
text_file.close()

In [16]:
# 训练word embedding模型
# sg=1，使用Skip-Gram模式（小规模语料适用）
word2vec_build_on_all_text = Word2Vec(LineSentence("./line_sents.txt"), size=400, window=5, sg=1, min_count=5)  # 训练模型
word2vec_build_on_all_text.wv.save_word2vec_format("./word2vec_build_on_all_text", binary=False)  # 保存模型

In [17]:
# 测试模型
# 输出与某个词相近的10个词
word2vec_build_on_all_text.wv.most_similar("机场")

[('换乘', 0.9867381453514099),
 ('K1', 0.9864612817764282),
 ('地级', 0.9829998016357422),
 ('城区', 0.9810256361961365),
 ('火车站', 0.9803483486175537),
 ('楚江', 0.9796959757804871),
 ('地级市', 0.9791610836982727),
 ('火车', 0.9775784015655518),
 ('桥', 0.9766429662704468),
 ('A3', 0.9757394194602966)]

## 二、分类模型
### 2.1 KNN模型操作练习
使用Sklearn自带的鸢尾花分类案例练习KNN模式

In [18]:
# 加载鸢尾花数据集
iris = sklearn.datasets.load_iris()
# 打印鸢尾花数据集的说明信息
print(iris["DESCR"])

.. _iris_dataset:

Iris plants dataset
--------------------

**Data Set Characteristics:**

    :Number of Instances: 150 (50 in each of three classes)
    :Number of Attributes: 4 numeric, predictive attributes and the class
    :Attribute Information:
        - sepal length in cm
        - sepal width in cm
        - petal length in cm
        - petal width in cm
        - class:
                - Iris-Setosa
                - Iris-Versicolour
                - Iris-Virginica
                
    :Summary Statistics:

    ============== ==== ==== ======= ===== ====================
                    Min  Max   Mean    SD   Class Correlation
    ============== ==== ==== ======= ===== ====================
    sepal length:   4.3  7.9   5.84   0.83    0.7826
    sepal width:    2.0  4.4   3.05   0.43   -0.4194
    petal length:   1.0  6.9   3.76   1.76    0.9490  (high!)
    petal width:    0.1  2.5   1.20   0.76    0.9565  (high!)
    ============== ==== ==== ======= ===== ===========

In [19]:
# 把数据集划分成训练集与测试集
X, y = iris.data, iris.target
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y, test_size=0.3)

X是 \[sample, feature\] 形式的数组，y是 target 数组

In [20]:
# 查看X数组的结构
X_train[:10]  # 显示前10个

array([[5.1, 3.7, 1.5, 0.4],
       [7.2, 3.2, 6. , 1.8],
       [5.7, 2.8, 4.5, 1.3],
       [6.3, 2.3, 4.4, 1.3],
       [5.8, 2.8, 5.1, 2.4],
       [6.7, 3. , 5.2, 2.3],
       [6.3, 3.3, 4.7, 1.6],
       [5.6, 2.8, 4.9, 2. ],
       [5. , 3.2, 1.2, 0.2],
       [6.4, 2.9, 4.3, 1.3]])

In [21]:
# 查看y数组的结构
y_train

array([0, 2, 1, 1, 2, 2, 1, 2, 0, 1, 1, 0, 1, 2, 2, 2, 1, 2, 1, 0, 0, 1,
       0, 2, 2, 0, 1, 0, 0, 1, 1, 0, 2, 2, 1, 2, 1, 2, 1, 0, 0, 1, 1, 1,
       2, 0, 1, 0, 1, 1, 1, 2, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 2,
       2, 0, 0, 2, 1, 0, 2, 0, 2, 1, 0, 2, 1, 0, 1, 0, 2, 1, 0, 1, 2, 0,
       1, 1, 0, 1, 1, 2, 2, 2, 2, 1, 0, 2, 1, 2, 2, 2, 2])

可以看到，在鸢尾花例中，分类目标（target）共有三种（0, 1, 2）

In [22]:
# 引入KNN模型
from sklearn.neighbors import KNeighborsClassifier
# 实例化一个KNN模型
knn = KNeighborsClassifier()

In [23]:
# 填充训练数据进行训练
knn.fit(X=X_train, y=y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

In [24]:
# 使用模型预测
knn.predict(X_test)

array([1, 2, 2, 0, 2, 1, 0, 0, 2, 2, 1, 2, 0, 0, 0, 2, 0, 2, 2, 0, 2, 2,
       1, 1, 0, 2, 1, 0, 0, 0, 0, 1, 2, 0, 2, 1, 2, 0, 0, 0, 2, 2, 2, 2,
       0])

### 2.2 Naive Bayes模型操作练习

In [25]:
from sklearn.datasets import fetch_20newsgroups

categories = ['alt.atheism', 'soc.religion.christian', \
              'comp.graphics', 'sci.med']

# 加载sklearn自带的新闻语料
twenty_train = fetch_20newsgroups(subset='train',
    categories=categories, shuffle=True, random_state=42)
twenty_test = fetch_20newsgroups(subset='test',
    categories=categories, shuffle=True, random_state=42)

In [26]:
# 加载sklearn自带的向量化工具
from sklearn.feature_extraction.text import CountVectorizer
# 构造向量化工具
count_vect = CountVectorizer()

# 拟合模型
X_train_counts = count_vect.fit_transform(twenty_train.data)
X_train_counts.shape

(2257, 35788)

上面shape字段的元组(2257, 35788)表示共有2257篇训练文章，词典的大小为35788维（有这么多个不重复的词）

In [27]:
# 构建TF-TDF特征
# 加载sklearn自带的TF-TDF构造器
from sklearn.feature_extraction.text import TfidfTransformer

# 用训练数据进行拟合
tf_transformer = TfidfTransformer().fit(X_train_counts)
# 构建TF-IDF特征
X_train_tf = tf_transformer.transform(X_train_counts)

len(X_train_tf.data)

365886

TF-IDF特征将每个词在当前文档中的重要程度给计算出来了，而且归一化

In [28]:
# 构建朴素贝叶斯分类器
from sklearn.naive_bayes import MultinomialNB
# 使用特征集（X）和目标（target）训练/拟合一个朴素贝叶斯分类器
clf = MultinomialNB().fit(X_train_tf, twenty_train.target)

clf  # 分类器

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

使用训练好的模型预测测试集

先将测试集的数据也转化成TF-IDF特征的形式，且需要用刚才构造的转换器

而转化TF-IDF之前，还要先将语料向量化，要使用之前构造的转换器

In [29]:
# 预测测试
# 将测试集数据向量化并转化成TF-IDF的形式
test_tf = tf_transformer.transform(count_vect.transform(twenty_test.data))

# 使用模型预测
predicted = clf.predict(test_tf)

predicted

array([2, 2, 3, ..., 2, 2, 1])

predicted集合中即为预测的结果，每个分量代表一个预测分类

In [30]:
# 使用sklearn自带的评价工具来评价分类的结果
from sklearn import metrics  # 引入评价工具

print(metrics.classification_report(twenty_test.target, predicted, target_names=twenty_test.target_names))

                        precision    recall  f1-score   support

           alt.atheism       0.97      0.60      0.74       319
         comp.graphics       0.96      0.89      0.92       389
               sci.med       0.97      0.81      0.88       396
soc.religion.christian       0.65      0.99      0.78       398

              accuracy                           0.83      1502
             macro avg       0.89      0.82      0.83      1502
          weighted avg       0.88      0.83      0.84      1502



评价结果分别展示了四个分类的准确率、召回率等指标

### 2.3 使用表2的语料训练KNN模型（TF-IDF）

In [31]:
# 构造文本列表
# 列表的格式为“单条评论词链表”的列表
# train_text = [comm_dict_2[row[0]].seg_topic + comm_dict_2[row[0]].seg_detail for row in comments]
train_text = [row[2] + row[4] for row in comments]

In [32]:
# 尝试使用TF-IDF构造特征
count_vect = CountVectorizer()
train_count = count_vect.fit_transform(train_text)
train_count

<495x15430 sparse matrix of type '<class 'numpy.int64'>'
	with 16320 stored elements in Compressed Sparse Row format>

In [33]:
from sklearn.feature_extraction.text import TfidfTransformer
# 构建TF-IDF转换器
tf_transformer = TfidfTransformer().fit(train_count)

In [34]:
# 把训练集转化成tf-idf
train_tf = tf_transformer.transform(train_count)

In [35]:
# 提取训练集中的全部标签
train_target_names = list(set(list(map(lambda x: x[5], comments))))
train_target_names

['教育文体', '城乡建设', '商贸旅游', '劳动和社会保障', '交通运输', '环境保护', '卫生计生']

In [36]:
# 将标签转换为数字序号
train_target = [train_target_names.index(comm_dict_2[row[0]].fir_lev_label) for row in comments]
train_target[98] = 5

In [37]:
# 从训练集中分出一部分作为测试集
train_tf, test_tf, train_target, test_target = sklearn.model_selection.train_test_split(train_tf, train_target, test_size=0.3)

In [38]:
# 引入KNN模型
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier()
# 训练模型
knn.fit(train_tf, train_target)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

In [39]:
# 预测语料的分类
test_predicted = knn.predict(test_tf)

In [40]:
test_tf

<149x15430 sparse matrix of type '<class 'numpy.float64'>'
	with 4228 stored elements in Compressed Sparse Row format>

In [41]:
# 评价模型
from sklearn import metrics 
print(metrics.classification_report(test_target, test_predicted, target_names=train_target_names))

              precision    recall  f1-score   support

        教育文体       0.21      0.16      0.18        31
        城乡建设       0.20      0.17      0.18        30
        商贸旅游       0.00      0.00      0.00        19
     劳动和社会保障       0.55      0.17      0.26        35
        交通运输       0.10      0.56      0.17        16
        环境保护       0.00      0.00      0.00         6
        卫生计生       0.00      0.00      0.00        12

    accuracy                           0.17       149
   macro avg       0.15      0.15      0.11       149
weighted avg       0.22      0.17      0.15       149



/opt/conda/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### 2.4 从Word2Vec向量得到文档向量
求和求平均得到文档向量：
对每一篇文章的词向量进行求和，然后除以词数量，得到文档向量

In [42]:
# 加载之前训练的Word2Vec模型
word2vec_model = gensim.models.KeyedVectors.load_word2vec_format("/kaggle/working/word2vec_e2_model", binary=False)

In [43]:
# 定义计算文档向量的方法
def get_doc_vec(doc: list, model):
    """计算文档向量"""
    ignore = ["\t", " ", "\n"]
    words = [word for word in doc if word not in ignore]
    # 所有词向量求和并除以词数量
    words_num = len(words)
    
    print(words_num)
    
    vec_sum = 0
    for word in words:
        try:
            vec_sum += model[word]
        except KeyError:
            words_num -= 1  # todo::是否应该跳过不在词典中的词？为什么会很多词不在词典中？
            continue
            
    print(words_num)

    return vec_sum / words_num

In [44]:
# 只有大概一半非空格词在词典中
for row in comments:
    c = comm_dict_2[row[0]]
    doc = c.seg_topic + c.seg_detail
    get_doc_vec(doc, word2vec_build_on_all_text)
    print()

88
45

79
30

501
375

100
65

81
33

115
46

123
63

82
29

112
56

1735
1296

1735
1296

83
28

85
47

154
94

90
48

63
23

86
32

125
69

68
26

100
58

74
26

143
73

160
84

106
38

182
117

50
12

95
44

192
122

56
19

318
209

501
401

109
63

227
105

546
252

92
30

75
28

107
66

796
663

184
95

78
33

340
184

524
343

299
186

407
250

77
28

353
205

241
155

450
258

589
451

114
69

77
36

171
92

635
318

261
152

88
37

144
79

120
66

579
354

144
85

133
87

87
40

348
239

550
378

533
413

236
141

91
41

193
124

465
291

1317
1084

99
50

456
327

443
350

939
746

90
48

188
128

94
51

178
113

79
30

58
23

76
39

48
17

137
88

65
27

217
152

149
87

631
409

76
35

98
49

73
30

89
52

70
32



/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:14: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


279
185

354
204

92
29

85
38

171
93

188
104

417
346

60
24

156
99

891
422

353
258

66
28

71
26

92
49

163
86

123
68

139
50

304
171

133
65

254
170

112
42

348
234

711
463

1066
747

512
349

739
472

74
27

304
166

172
88

225
104

118
76

111
53

167
89

171
103

799
622

274
161

626
355

179
105

190
113

294
187

108
69

420
320

350
225

117
64

331
166

145
77

863
535

233
143

131
73

233
127

270
174

69
32

90
48

174
98

97
56

61
26

140
63

99
41

170
107

70
24

345
199

211
115

61
13

127
61

124
76

129
71

130
69

468
271

140
83

140
82

84
36

68
23

68
26

107
43

56
13

74
32

156
95

88
41

118
60

581
387

119
51

84
34

386
145

76
37

152
89

128
70

81
39

119
64

176
87

68
28

272
213

95
47

156
86

128
61

119
53

360
234

110
55

179
102

119
60

553
358

122
67

107
48

146
90

136
76

172
99

404
279

75
38

746
437

131
73

140
82

62
26

151
95

216
147

121
80

58
22

329
250

86
45

77
35

336
238

331
199

331
150

635
427

149
89

原先使用的是仅从表2语料构造的词嵌入模型，换成从三个表构造的模型（word2vec_build_on_all_text）后，分类器的准确率有所提升

In [45]:
# 计算所有评论的文档向量
comms_vec = [get_doc_vec(comm_dict_2[row[0]].seg_topic + comm_dict_2[row[0]].seg_detail, word2vec_build_on_all_text) for row in comments]

88
45
79
30
501
375
100
65
81
33
115
46
123
63
82
29
112
56
1735
1296
1735
1296
83
28
85
47
154
94
90
48
63
23
86
32
125
69
68
26
100
58
74
26
143
73
160
84
106
38
182
117
50
12
95
44
192
122
56
19
318
209
501
401
109
63
227
105
546
252
92
30
75
28
107
66
796
663
184
95
78
33
340
184
524
343
299
186
407
250
77
28
353
205
241
155
450
258
589
451
114
69
77
36
171
92
635
318
261
152
88
37
144
79
120
66
579
354
144
85
133
87
87
40
348
239
550
378
533
413
236
141
91
41
193
124
465
291
1317
1084
99
50
456
327
443
350
939
746
90
48
188
128
94
51
178
113
79
30
58
23
76
39
48
17
137
88
65
27
217
152
149
87
631
409
76
35
98
49
73
30
89
52
70
32
279
185
354
204
92
29
85
38
171
93
188
104
417
346
60
24
156
99
891
422
353
258
66
28
71
26
92
49
163
86
123
68
139
50
304
171
133
65
254
170
112
42
348
234
711
463
1066
747
512
349
739


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:14: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


472
74
27
304
166
172
88
225
104
118
76
111
53
167
89
171
103
799
622
274
161
626
355
179
105
190
113
294
187
108
69
420
320
350
225
117
64
331
166
145
77
863
535
233
143
131
73
233
127
270
174
69
32
90
48
174
98
97
56
61
26
140
63
99
41
170
107
70
24
345
199
211
115
61
13
127
61
124
76
129
71
130
69
468
271
140
83
140
82
84
36
68
23
68
26
107
43
56
13
74
32
156
95
88
41
118
60
581
387
119
51
84
34
386
145
76
37
152
89
128
70
81
39
119
64
176
87
68
28
272
213
95
47
156
86
128
61
119
53
360
234
110
55
179
102
119
60
553
358
122
67
107
48
146
90
136
76
172
99
404
279
75
38
746
437
131
73
140
82
62
26
151
95
216
147
121
80
58
22
329
250
86
45
77
35
336
238
331
199
331
150
635
427
149
89
124
69
64
24
100
53
91
46
133
73
109
57
150
81
461
292
96
58
85
40
222
129
136
89
505
411
177
109
344
218
236
157
127
76
131
72
113
68
313
169
86
38
157
92
62
20
128
61
166
100
191
125
309
222
398
289
268
155
97
48
154
85
260
165
65
23
300
187
77
29
180
96
221
127
64
31
78
33
64
24
80
40
253
120
72
31
69
2

虽然不知道为什么词典中少了很多词语，但总算是得到了针对每条评论的“文档向量”

In [46]:
# 表2中涉及的所有一级标签
target_names = list(set([row[5] for row in comments]))
target_names

['教育文体', '城乡建设', '商贸旅游', '劳动和社会保障', '交通运输', '环境保护', '卫生计生']

In [47]:
# 将表2中标注的所有一级标签转化成数字表示（target_names中的index）
targets = [target_names.index(row[5]) for row in comments]

targets[:10]  # 展示前十个

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

In [48]:
# 分割训练集和测试集
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(comms_vec, targets, test_size=0.3)

In [49]:
# 引入KNN模型
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(leaf_size=50, n_neighbors=15)

In [50]:
# 训练knn模型
knn.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=50, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=15, p=2,
                     weights='uniform')

In [51]:
predicted = knn.predict(X_test)

In [52]:
# 评价模型
from sklearn import metrics 
print(metrics.classification_report(y_test, predicted, target_names=target_names))

              precision    recall  f1-score   support

        教育文体       0.57      0.52      0.54        33
        城乡建设       0.28      0.52      0.36        23
        商贸旅游       0.31      0.33      0.32        12
     劳动和社会保障       0.77      0.77      0.77        31
        交通运输       0.40      0.21      0.28        19
        环境保护       0.33      0.23      0.27        13
        卫生计生       0.46      0.33      0.39        18

    accuracy                           0.47       149
   macro avg       0.45      0.42      0.42       149
weighted avg       0.49      0.47      0.47       149



### 2.5 使用朴素贝叶斯模型
贝叶斯模型参数少、训练快，在测试中表现优秀，在大数据集下应该会有更好的表现

In [53]:
# 构造向量化工具
count_vect = CountVectorizer()
comments = read_xl_by_line(path_cc + "/xls/e2.xlsx")  # 留言文本
stop_words = load_word_list(path_cc + "/special-words/stop_words.txt")  # 停用词
comm_dict_2 = generate_comm_dict(comments, True, stop_words)

line_sents = [comm_dict_2[row[0]].seg_topic + comm_dict_2[row[0]].seg_detail for row in comments]
sents = list(map(lambda x: " ".join(x), line_sents))

# 表2中涉及的所有一级标签
target_names = list(set([row[5] for row in comments]))
# 将表2中标注的所有一级标签转化成数字表示（target_names中的index）
targets = [target_names.index(row[5]) for row in comments]

In [54]:
# 分割训练集和测试集
x_train, x_test, y_train, y_test \
    = sklearn.model_selection.train_test_split(sents, targets, test_size=0.3)

x_train_counts = count_vect.fit_transform(x_train)  # 拟合模型

In [55]:
# 构建TF-TDF特征
tf_transformer = TfidfTransformer().fit(x_train_counts)
# 构建TF-IDF特征
x_train_tf = tf_transformer.transform(x_train_counts)

# 使用特征集（X）和目标（target）训练/拟合一个朴素贝叶斯分类器
clf = MultinomialNB()
clf.fit(x_train_tf.toarray(), y_train)

test_tf = tf_transformer.transform(count_vect.transform(x_test))

predicted = clf.predict(test_tf.toarray())  # 分类器

print(sklearn.metrics.classification_report(y_true=y_test, y_pred=predicted, target_names=target_names))

              precision    recall  f1-score   support

        教育文体       0.82      0.84      0.83        32
        城乡建设       0.59      0.72      0.65        36
        商贸旅游       0.00      0.00      0.00        11
     劳动和社会保障       0.58      0.97      0.73        33
        交通运输       0.86      0.46      0.60        13
        环境保护       1.00      0.25      0.40         8
        卫生计生       1.00      0.50      0.67        16

    accuracy                           0.68       149
   macro avg       0.69      0.54      0.55       149
weighted avg       0.68      0.68      0.64       149



/opt/conda/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
